In [55]:
import pandas as pd
import h5py, json

In [143]:
infile = 's5-series_json.dir/GSE71585-GPL13112.json'

In [144]:
# Get dataset json
with open(infile) as openfile:
    dataset = json.loads(openfile.read())

# Get dataset accession
dataset_accession = os.path.basename(infile).split('-')[0]


In [145]:
from sqlalchemy import create_engine, MetaData
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy.orm import sessionmaker


In [146]:
engine = create_engine(os.environ['SQLALCHEMY_DATABASE_URI']+'-dev?charset=utf8')
Session = sessionmaker(bind=engine)
metadata = MetaData()
metadata.reflect(bind=engine)
tables = metadata.tables


In [100]:
def upload_data(data, keys, table):

    return engine.execute(tables[table].insert({x: data[x] for x in keys}))

In [101]:
a = upload_data(data=dataset, keys=['title', 'summary', 'date', 'accession'], table='dataset')
a

In [116]:
engine.execute(tables['platform'].select().where(tables['platform'].columns['gpl'] == 'GPL13112')).fetchall()[0]['id']

1

In [104]:
a.lastrowid

4

In [147]:

# Create sample dataframe
sample_dataframe = pd.DataFrame(dataset['samples']).T.reset_index().rename(columns={'index': 'accession', 'Sample Title': 'title'})[['accession', 'title']]

# Add foreign keys
sample_dataframe['dataset_fk'] = dataset['dataset_id']
sample_dataframe['platform_fk'] = dataset['platform_id']

# Append
sample_dataframe.to_sql('sample_new', if_exists='append')

KeyError: 'dataset_id'

In [152]:
dataset['sample_ids'] = pd.DataFrame(engine.execute(tables['sample_new'].select()).fetchall()).rename(columns={0: 'sample_fk', 1: 'accession'})[['sample_fk', 'accession']]
dataset['sample_ids']

,sample_fk,accession
0,1,GSM1839194
1,2,GSM1839202
2,3,GSM1839262
3,4,GSM1839263
4,5,GSM1839264
5,6,GSM1839265
6,7,GSM1839266
7,8,GSM1839267
8,9,GSM1839268
9,10,GSM1839269


In [150]:
pd.melt(pd.DataFrame(dataset['samples']).T.reset_index(), id_vars='index').merge(dataset['sample_ids'], left_on='index', right_on='accession')

,index,variable,value
0,GSM1839192,Sample Title,Calb2_tdTpositive_cell_1
1,GSM1839193,Sample Title,Calb2_tdTpositive_cell_2
2,GSM1839195,Sample Title,Calb2_tdTpositive_cell_4
3,GSM1839196,Sample Title,Calb2_tdTpositive_cell_5
4,GSM1839197,Sample Title,Calb2_tdTpositive_cell_6
5,GSM1839198,Sample Title,Calb2_tdTpositive_cell_7
6,GSM1839199,Sample Title,Calb2_tdTpositive_cell_8
7,GSM1839200,Sample Title,Calb2_tdTpositive_cell_9
8,GSM1839201,Sample Title,Calb2_tdTpositive_cell_10
9,GSM1839203,Sample Title,Calb2_tdTpositive_cell_12
